In [1]:
from tensorflow.python.keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import random
import time

Using TensorFlow backend.


In [12]:
BATCH_SIZE = 256
SAMPLES_PER_EPOCH = 50000
EPOCH_NUM = 500

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)


def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b
  
def get_batch(features,labels,batch_size): 
    num_images = features.shape[0]
    idx = np.random.choice(num_images,
                           size=batch_size,
                           replace=False)
    features_batch = features[idx, :, :, :]
    labels_batch = labels[idx, :]
    return features_batch, labels_batch

In [5]:
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()

num_train, img_rows, img_cols, img_channels =  train_features.shape
num_test =  test_features.shape[0]
num_classes = np.unique(train_labels).shape[0]

train_features = train_features.astype('float32')/np.max(train_features)
test_features = test_features.astype('float32')/np.max(test_features)


train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

print(train_features.shape, train_labels.shape)

(50000, 32, 32, 3) (50000, 10)


In [6]:
x = tf.placeholder(tf.float32 , shape=[None , img_rows,img_cols,img_channels]) 
y_ = tf.placeholder(tf.float32 , shape=[None, 10]) 
keep_prob = tf.placeholder(tf.float32)

test_features, test_labels = get_batch(test_features,test_labels,500)

In [8]:
with tf.name_scope('conv_1'): 
    conv1 = conv_layer(x , shape=[3, 3, img_channels, 64])
    conv1_pool = max_pool_2x2( conv1 )

with tf.name_scope('conv_2'): 
    conv2 = conv_layer(conv1_pool , shape=[3, 3, 64, 128])
    conv2_pool = max_pool_2x2 ( conv2 )

with tf.name_scope('conv_3'): 
    conv3 = conv_layer(conv2_pool , shape=[3, 3, 128, 256] )
    conv3_pool = max_pool_2x2 ( conv3 )
    conv3_flat = tf.contrib.layers.flatten(conv3_pool) 

with tf.name_scope('full_1'): 
    full_1 = tf.nn.relu(full_layer(conv3_flat , 512))

with tf.name_scope('dropout'): 
    full1_drop = tf.nn.dropout(full_1 , keep_prob=keep_prob) 

with tf.name_scope('activations'): 
    y_conv = full_layer(full1_drop , 10) 
    tf.summary.scalar('cross_entropy_loss',y_conv)	

with tf.name_scope('cross'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv , labels=y_)) 

In [10]:
train_step = tf.train.AdagradOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv , 1), tf.argmax(y_, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
print(test_features.shape, test_labels.shape)

(500, 32, 32, 3) (500, 10)


In [13]:
 with tf.device("/gpu:0"):   
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        start_time = time.time()
        
        for j in range(EPOCH_NUM):
            for i in range(SAMPLES_PER_EPOCH // BATCH_SIZE):
                batch_trainf, batch_trainl = get_batch(train_features,train_labels,BATCH_SIZE)
                sess.run(train_step , feed_dict={x: batch_trainf, y_: batch_trainl, keep_prob: 0.5}) 
            batch_trainf, batch_trainl = get_batch(test_features,test_labels,32)
            train_accuracy = sess.run(accuracy , feed_dict={x: batch_trainf, y_: batch_trainl, keep_prob: 1.0}) 
            print("time {}, epoch {}, training accuracy {}".format(time.time() - start_time, j, train_accuracy)) 

        test_accuracy = np.mean([sess.run(accuracy , feed_dict={x:test_features, y_:test_labels, keep_prob:1.0})])
        print("test accuracy: {}".format(test_accuracy))

time 11.172466278076172, epoch 0, training accuracy 0.40625
time 22.15634298324585, epoch 1, training accuracy 0.5625
time 33.11371684074402, epoch 2, training accuracy 0.375
time 44.08511161804199, epoch 3, training accuracy 0.40625
time 55.056150913238525, epoch 4, training accuracy 0.40625
time 66.04810214042664, epoch 5, training accuracy 0.5
time 77.04403734207153, epoch 6, training accuracy 0.34375
time 88.06166553497314, epoch 7, training accuracy 0.4375
time 99.06902694702148, epoch 8, training accuracy 0.3125
time 110.0735695362091, epoch 9, training accuracy 0.5
time 121.08499360084534, epoch 10, training accuracy 0.46875
time 132.09572005271912, epoch 11, training accuracy 0.375
time 143.10369038581848, epoch 12, training accuracy 0.5
time 154.10197043418884, epoch 13, training accuracy 0.5
time 165.12351536750793, epoch 14, training accuracy 0.5625
time 176.14336276054382, epoch 15, training accuracy 0.4375
time 187.17725706100464, epoch 16, training accuracy 0.5
time 198.1

time 1575.1645748615265, epoch 138, training accuracy 0.53125
time 1586.6145315170288, epoch 139, training accuracy 0.46875
time 1597.975317955017, epoch 140, training accuracy 0.59375
time 1609.4048895835876, epoch 141, training accuracy 0.5
time 1620.8717620372772, epoch 142, training accuracy 0.71875
time 1632.1949820518494, epoch 143, training accuracy 0.53125
time 1643.54576253891, epoch 144, training accuracy 0.5
time 1655.054877281189, epoch 145, training accuracy 0.65625
time 1666.5140256881714, epoch 146, training accuracy 0.625
time 1677.9051296710968, epoch 147, training accuracy 0.46875
time 1689.4019920825958, epoch 148, training accuracy 0.625
time 1700.8471355438232, epoch 149, training accuracy 0.4375
time 1712.2862186431885, epoch 150, training accuracy 0.5625
time 1723.6990542411804, epoch 151, training accuracy 0.71875
time 1735.0588359832764, epoch 152, training accuracy 0.5625
time 1746.6235029697418, epoch 153, training accuracy 0.65625
time 1758.060703277588, epo

time 3131.4493613243103, epoch 274, training accuracy 0.75
time 3142.8611516952515, epoch 275, training accuracy 0.71875
time 3154.2347917556763, epoch 276, training accuracy 0.5625
time 3165.798013687134, epoch 277, training accuracy 0.375
time 3177.218919277191, epoch 278, training accuracy 0.59375
time 3188.591910123825, epoch 279, training accuracy 0.59375
time 3199.9041063785553, epoch 280, training accuracy 0.5625
time 3211.4611434936523, epoch 281, training accuracy 0.5625
time 3222.8993656635284, epoch 282, training accuracy 0.53125
time 3234.303685903549, epoch 283, training accuracy 0.53125
time 3245.794689655304, epoch 284, training accuracy 0.65625
time 3257.315860271454, epoch 285, training accuracy 0.59375
time 3268.7815442085266, epoch 286, training accuracy 0.625
time 3280.3579382896423, epoch 287, training accuracy 0.65625
time 3291.805622816086, epoch 288, training accuracy 0.5
time 3303.2761204242706, epoch 289, training accuracy 0.59375
time 3314.5806596279144, epoc

time 4685.551441669464, epoch 410, training accuracy 0.625
time 4696.933883905411, epoch 411, training accuracy 0.59375
time 4708.341626167297, epoch 412, training accuracy 0.65625
time 4719.702955961227, epoch 413, training accuracy 0.625
time 4731.134139060974, epoch 414, training accuracy 0.53125
time 4742.55773639679, epoch 415, training accuracy 0.625
time 4754.153601646423, epoch 416, training accuracy 0.5625
time 4765.823804616928, epoch 417, training accuracy 0.65625
time 4777.567023992538, epoch 418, training accuracy 0.65625
time 4789.292713880539, epoch 419, training accuracy 0.625
time 4800.71041727066, epoch 420, training accuracy 0.5625
time 4812.325362920761, epoch 421, training accuracy 0.71875
time 4824.1120002269745, epoch 422, training accuracy 0.6875
time 4835.791969060898, epoch 423, training accuracy 0.59375
time 4848.466763973236, epoch 424, training accuracy 0.5625
time 4860.570207118988, epoch 425, training accuracy 0.65625
time 4872.290109872818, epoch 426, tr